In [1]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [2]:
#del sys.modules["utilities"]
from utilities import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [5]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))

df_icu_graph_features = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_icu_graph_features.pkl','rb'))

df_feats_ppg_train_all = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_train_separate.pkl','rb'))
df_feats_ppg_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_val_separate.pkl','rb'))
df_feats_ppg_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_test.pkl','rb'))


Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [6]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [7]:
df_icu_graph_features_train = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_val = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_test = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

len(df_icu_graph_features_train)+len(df_icu_graph_features_val)+len(df_icu_graph_features_test)-len(df_icu_graph_features)

0

In [8]:
df_feats_clinical_ppg_icug_train = df_feats_clinical[df_feats_clinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_train_all,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_train,how='left',on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_ppg_icug_val = df_feats_clinical[df_feats_clinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_ppg_icug_test = df_feats_clinical[df_feats_clinical['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_clinical_ppg_icug_train)+len(df_feats_clinical_ppg_icug_val)+len(df_feats_clinical_ppg_icug_test)-len(df_feats_clinical)

0

In [9]:
X_train = df_feats_clinical_ppg_icug_train
X_val = df_feats_clinical_ppg_icug_val
X_test = df_feats_clinical_ppg_icug_test

y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

In [10]:
X_train_val = pd.concat([X_train,X_val])
y_train_val = pd.concat([y_train,y_val])

In [11]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 2958),
 (13665, 2958),
 (122983, 2958),
 (13664, 2958),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [12]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [13]:
del sys.modules["utilities"]
from utilities import *

In [14]:
experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[666])

100%|███████████████████████████████████████| 1/1 [12:26:56<00:00, 44816.82s/it]


A. Scores on training set
avg XGBoost classifer: auc/  0.886  F1:  0.6153  precision:  0.766  recall:  0.5142  accuracy:  0.8421
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.8036  F1:  0.4823  precision:  0.6615  recall:  0.3795  accuracy:  0.7999
std MLP classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg logistic classifer: auc/  0.8381  F1:  0.6064  precision:  0.5134  recall:  0.7406  accuracy:  0.7638
std logistic classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
****************************************************

B. Scores on test set
avg XGBoost classifer: auc/  0.8262  F1:  0.4936  precision:  0.7509  recall:  0.3676  accuracy:  0.7823
std XGBoost classifer: auc/  0.0  F1:  0.0  precision:  0.0  recall:  0.0  accuracy:  0.0
avg MLP classifer: auc/  0.7782  F1:  0.3712  precision:  0.7469  recall:  0.247  accuracy:  0.7585
std MLP classifer: auc/  0